# Preprocesamiento: Valores numéricos no disponibles (*missing values*)

Este notebook es una adaptación del [original de *Aurélien Gerón*](https://github.com/ageron/handson-ml3/blob/main/02_end_to_end_machine_learning_project.ipynb), de su libro: [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 3rd Edition. Aurélien Géron](https://www.oreilly.com/library/view/hands-on-machine-learning/9781098125967/)

## Pasos previos

In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

housing = pd.read_csv("./data/housing.csv") 

# Generación de conjuntos de entrenamiento y prueba mediante muestreo estratificado por ingreso medio
train_set, test_set = train_test_split(housing, test_size=0.2,
    stratify=pd.cut(housing["median_income"], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5]),
    random_state=42
    )

El **preprocesamiento** de los datos es una de las tareas más importantes en Machine Learning. Si los datos no están bien preparados, los algoritmos de *Machine Learning* no funcionarán correctamente. Primero vamos a separar los **predictores** de la **variable objetivo** (las **etiquetas**), ya que no necesariamente se aplicarán las mismas **transformaciones** a ambos.

In [34]:
housing = train_set.drop("median_house_value", axis=1) # Eliminamos la columna de la variable dependiente
housing_labels = train_set["median_house_value"].copy() # Guardamos la variable dependiente (etiquetas)

housing.head().T

,12655,15502,2908,14053,20496
longitude,-121.46,-117.23,-119.04,-117.13,-118.7
latitude,38.52,33.09,35.37,32.75,34.28
housing_median_age,29.0,7.0,44.0,24.0,27.0
total_rooms,3873.0,5320.0,1618.0,1877.0,3536.0
total_bedrooms,797.0,855.0,310.0,519.0,646.0
population,2237.0,2015.0,667.0,898.0,1837.0
households,706.0,768.0,300.0,483.0,580.0
median_income,2.1736,6.3373,2.875,2.2264,4.4964
ocean_proximity,INLAND,NEAR OCEAN,INLAND,NEAR OCEAN,<1H OCEAN


## Identificación de valores no disponibles

Como vimos al principio, la columna 'total_bedrooms' tiene valores no diponibles. Normalente se hablará como sinónimos de valores no disponibles, ***missing values***, *nulos* o ***na* (not available)**, aunque hay que tener cuidado de cómo se recogieron esos valores, ya que si hay dos tipos de valores (null y *string* vacío, por ejemplo) podría haber información implícita.
<!-- TODO: Revisar distinción entre na, null, missing, datos impuros.... -->

In [35]:
housing.isna().sum() # el método isnull() es un alias de isna()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        158
population              0
households              0
median_income           0
ocean_proximity         0
dtype: int64

In [36]:
null_rows_idx = housing.isnull().any(axis=1) # índices de las filas con valores nulos
housing.loc[null_rows_idx].head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
1606,-122.08,37.88,26.0,2947.0,NaN,825.0,626.0,2.9330,NEAR BAY
10915,-117.87,33.73,45.0,2264.0,NaN,1970.0,499.0,3.4193,<1H OCEAN
19150,-122.70,38.35,14.0,2313.0,NaN,954.0,397.0,3.7813,<1H OCEAN
4186,-118.23,34.13,48.0,1308.0,NaN,835.0,294.0,4.2891,<1H OCEAN
16885,-122.40,37.58,26.0,3281.0,NaN,1145.0,480.0,6.3580,NEAR OCEAN


## Eliminación de las filas con valores nulos (***Listwise deletion***)

Podemos simplemente eliminar aquellas instancias incompletas, aunque es problemático porque estamos eliminando información. Especialmente si hay muchos predictores (ya que para solventar el problema de ciertos nulos estamos perdiendo la información de las demás columas).

In [37]:
housing_option1 = housing.dropna(subset=["total_bedrooms"]) 
housing_option1.loc[null_rows_idx].head() # comprobamos que se han eliminado las filas con valores nulos

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity


También podríamos directamente eliminar toda fila que tenga un valor na en cualquier columna:

In [38]:
housing_option1b = housing.dropna(axis=0) # eliminamos las filas con valores nulos
housing_option1b.loc[null_rows_idx].head() # comprobamos que se han eliminado las filas con valores nulos 

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity


## Eliminación de la columna entera

Eliminar la columna entera es una opción si no es una variable importante, pero en este caso parece que sí lo es dado que, aunque esa *feature* no es la que más correlaciona directamente con la variable objetivo, es una de las dos con las que se calcula `bedrooms_ratio`, que sí es la segunda más correlacionada.

In [39]:
housing_option2 = housing.drop(columns="total_bedrooms")
housing_option2.loc[null_rows_idx].head()

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,ocean_proximity
1606,-122.08,37.88,26.0,2947.0,825.0,626.0,2.9330,NEAR BAY
10915,-117.87,33.73,45.0,2264.0,1970.0,499.0,3.4193,<1H OCEAN
19150,-122.70,38.35,14.0,2313.0,954.0,397.0,3.7813,<1H OCEAN
4186,-118.23,34.13,48.0,1308.0,835.0,294.0,4.2891,<1H OCEAN
16885,-122.40,37.58,26.0,3281.0,1145.0,480.0,6.3580,NEAR OCEAN


Las filas siguen ahí en este caso porque los índices con nulos se buscaron antes. Si buscamos nulos ahora en housing_option2, no los encontraremos.

In [40]:
housing_option2.isnull().any(axis=None) # comprobamos que no hay valores nulos en el dataset

False

También podríamos eliminar directamente todas las columnas con nulos:

In [41]:
housing.dropna(axis=1).isnull().any(axis=None)

False

## Imputación de algún valor (la mediana en este caso)

La **imputación** de un determinado valor (como cero, la media o la mediana) a esos campos no disponibles es una opción si creemos que los valores no disponibles no responden a ninguna causa concreta, y no sesgan la distribución de la variable<!-- TODO: Revisar Different types of missing data (MCAR, MAR, MNAR) -->.

La imputación de la media (***mean***) es más sensible a los **valores atípicos**, ya que un valor extremo puede afectar mucho a la media. La mediana (***median***) es más robusta a los valores extremos. La moda (***mode***) es el valor que más se repite, y es útil para variables categóricas, pero no tanto para variables continuas.



[<img src="img/mean_outliers.jpg" width="300">](https://www.kaggle.com/code/nareshbhat/outlier-the-silent-killer)

In [42]:
median = housing["total_bedrooms"].median()
housing_option3 = housing["total_bedrooms"].fillna(median)
housing_option3.loc[null_rows_idx].head()

1606     433.0
10915    433.0
19150    433.0
4186     433.0
16885    433.0
Name: total_bedrooms, dtype: float64

Ahora todas estas filas tienen en total_bedrooms el valor mediano de total_bedrooms.

La clase `SimpleImputer` de scikit-learn nos permite hacer esto de forma más sencilla. Creamos una instancia de `SimpleImputer` indicando que queremos imputar los valores nulos con la mediana, y luego usamos el método `fit()` para calcular la mediana de cada columna y el método `transform()` para aplicar la imputación a todas las columnas.

Vamos a ver cómo se aplicaría este método a todos los campos numéricos del dataframe (recordemos que 'ocean_proximity' es categorial -valores de texto-, y no se puede calcular la mediana de un texto).

In [43]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

In [44]:
housing_num = housing.select_dtypes(include=[np.number]) # seleccionamos las columnas numéricas

In [45]:
imputer.fit(housing_num) # calculamos la mediana de cada columna numérica
imputer.statistics_ # mediana de cada columna numérica

array([-118.51   ,   34.26   ,   29.     , 2119.     ,  433.     ,
       1164.     ,  408.     ,    3.54155])

Podemos comprobar que los valores son los mismos que calcula el método `median()` del dataframe.

In [46]:
housing_num.median().values

array([-118.51   ,   34.26   ,   29.     , 2119.     ,  433.     ,
       1164.     ,  408.     ,    3.54155])

In [47]:
housing_num_array_tr = imputer.transform(housing_num) # reemplazamos los valores nulos por la mediana
housing_num_array_tr

array([[-1.2146e+02,  3.8520e+01,  2.9000e+01, ...,  2.2370e+03,
         7.0600e+02,  2.1736e+00],
       [-1.1723e+02,  3.3090e+01,  7.0000e+00, ...,  2.0150e+03,
         7.6800e+02,  6.3373e+00],
       [-1.1904e+02,  3.5370e+01,  4.4000e+01, ...,  6.6700e+02,
         3.0000e+02,  2.8750e+00],
       ...,
       [-1.2272e+02,  3.8440e+01,  4.8000e+01, ...,  4.5800e+02,
         1.7200e+02,  3.1797e+00],
       [-1.2270e+02,  3.8310e+01,  1.4000e+01, ...,  1.2080e+03,
         5.0100e+02,  4.1964e+00],
       [-1.2214e+02,  3.9970e+01,  2.7000e+01, ...,  6.2500e+02,
         1.9700e+02,  3.1319e+00]])

`transform()` devuelve un array de NumPy, pero podríamos convertirlo de nuevo a un DataFrame de Pandas.

In [48]:
housing_tr = pd.DataFrame(housing_num_array_tr, columns=housing_num.columns, index=housing_num.index)
housing_tr.loc[null_rows_idx].head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
1606,-122.08,37.88,26.0,2947.0,433.0,825.0,626.0,2.9330
10915,-117.87,33.73,45.0,2264.0,433.0,1970.0,499.0,3.4193
19150,-122.70,38.35,14.0,2313.0,433.0,954.0,397.0,3.7813
4186,-118.23,34.13,48.0,1308.0,433.0,835.0,294.0,4.2891
16885,-122.40,37.58,26.0,3281.0,433.0,1145.0,480.0,6.3580


También se podría utilizar directamente el método `fit_transform()` de `SimpleImputer` para calcular el valor a imputar (con `fit()`) y aplicarla (con `transform()`) en un solo paso.

Y también se podría utilizar el método `.set_output(transform="pandas")` del imputador para que el resultado sea un DataFrame de Pandas.

Por tanto, todo el proceso detallado anteriormente se puede resumir en una sola línea de código:

In [51]:
housing_tr = SimpleImputer(strategy="median").set_output(transform="pandas").fit_transform(housing_num)
housing_tr.loc[null_rows_idx].head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
1606,-122.08,37.88,26.0,2947.0,433.0,825.0,626.0,2.9330
10915,-117.87,33.73,45.0,2264.0,433.0,1970.0,499.0,3.4193
19150,-122.70,38.35,14.0,2313.0,433.0,954.0,397.0,3.7813
4186,-118.23,34.13,48.0,1308.0,433.0,835.0,294.0,4.2891
16885,-122.40,37.58,26.0,3281.0,433.0,1145.0,480.0,6.3580


## Modelos predictivos para imputar valores

Existen métodos más avanzados como el uso de **módelos de predicción** (tratando la columna con valores nulos como la variable objetivo y el resto de columnas como *features*). Por ejemplo, podría utilizarse el algoritmo **K-Nearest Neighbors (KNN)** para predecir los valores nulos de 'total_bedrooms' basándonos en los registros sí etiquetados. SciKit-Learn tiene una clase `KNNImputer` que hace esto.